In [504]:
import pandas as pd

from ssl import create_default_context
ssl_context = create_default_context()

import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning, SubjectAltNameWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
requests.packages.urllib3.disable_warnings(SubjectAltNameWarning)

import json
import uuid

In [203]:
headers = {'Content-Type': 'application/json'}

# Get data

In [48]:
xls = pd.ExcelFile('../data/testdata.xlsx')
xls.sheet_names

['Formål GML',
 'Formål',
 'Ark1',
 'data-catalog-backend',
 'data-catalog-policies',
 'Category']

## Codelist

In [820]:
codelist_url = 'https://datakatalog.nais.oera-q.local/backend/codelist'

In [821]:
df_codelist = pd.read_json(codelist_url).reset_index()

In [822]:
df_codelist.head()

,index,PURPOSE,CATEGORY,PROVENANCE
0,ALDERSPENSJON,Behandle og vurdere rett til alderspensjon som...,Motorvognopplysninger,NaN
1,ARBEIDSAVKLARINGSPENGER (AAP),Behandle og vurdere rett til arbeidsavklarings...,Opplysninger om reisevei,NaN
2,AVTALEFESTET PENSJON (AFP) FOR PRIVAT SEKTOR,AFP i privat sektor: Behandle og vurdere om vi...,Adresse og kontaktopplysninger,NaN
3,AVTALEFESTET PENSJON (AFP) FOR STATLIG OG KOMM...,Yte avtalefestet pensjon til 62-65-åringer som...,Arbeidsforhold,NaN
4,BARNEBIDRAG,Behandle krav om bidrag som skal sikre barnet ...,Andre økonomiske opplysninger,NaN


In [617]:
df_current_purpose = df_codelist[['index','PURPOSE']].dropna()
df_current_purpose.columns=['code','list']
df_current_purpose.head()

,code,list
0,ALDERSPENSJON,Behandle og vurdere rett til alderspensjon som...
1,ARBEIDSAVKLARINGSPENGER (AAP),Behandle og vurdere rett til arbeidsavklarings...
2,AVTALEFESTET PENSJON (AFP) FOR PRIVAT SEKTOR,AFP i privat sektor: Behandle og vurdere om vi...
3,AVTALEFESTET PENSJON (AFP) FOR STATLIG OG KOMM...,Yte avtalefestet pensjon til 62-65-åringer som...
4,BARNEBIDRAG,Behandle krav om bidrag som skal sikre barnet ...


# Purpose

In [618]:
sheet=xls.sheet_names[2]
df_purpose = pd.read_excel(xls, sheet)
df_purpose.head(1)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Code,Unnamed: 4,Description
0,"""MOTORVOGNOPPLYSNINGER"": ""Motorvognopp...",32,"""MOTORVOGNOPPLYSNINGER",MOTORVOGNOPPLYSNINGER,"Motorvognopplysninger"",",Motorvognopplysninger


### Clean list - delete all current items

In [619]:
for index, row in df_current_purpose.iterrows():
    delete_url = f"{codelist_url}/purpose/{row['code']}"
    requests.delete(delete_url)

### Populate list with new data

In [631]:
sheet=xls.sheet_names[1]
df_formål = pd.read_excel(xls, sheet, encoding='latin1')
df_formål = df_formål[['Forslag til endelig formålskode', 'Formålsbeskrivelse']]
df_formål.columns=['code', 'description']
df_formål['code'] = df_formål['code'].apply(lambda x: x.upper())
df_formål['list']='purpose'
df_formål = df_formål.append({'code': 'NOT AVAILABLE', 'description': 'Not Available -Verdi mangler', 'list': 'purpose'}, ignore_index=True)
df_formål.tail()

,code,description,list
33,UFØRETRYGD,Behandle og vurdere rett til uføretrygd som sk...,purpose
34,VENTELØNN,Behandle og vurdere rett til ventelønn som ska...,purpose
35,YRKESSKADE- OG SYKDOM,Behandle og vurdere rett til ytelser ved yrkes...,purpose
36,YTELSER TIL FAMILIEPLEIER,Behandle og vurdere rett til ytelser til famil...,purpose
37,NOT AVAILABLE,Not Available -Verdi mangler,purpose


In [632]:
data = df_formål.to_json(orient='records')

In [633]:
payload =json.dumps(json.loads(data))

In [634]:
requests.post(codelist_url, headers=headers, data=payload)

<Response [201]>

# Categories

### Delete existing

In [649]:
df_current_purpose = df_codelist[['index','CATEGORY']].dropna()
df_current_purpose.head()

,index,CATEGORY
0,ALDERSPENSJON,Motorvognopplysninger
1,ARBEIDSAVKLARINGSPENGER (AAP),Opplysninger om reisevei
2,AVTALEFESTET PENSJON (AFP) FOR PRIVAT SEKTOR,Adresse og kontaktopplysninger
3,AVTALEFESTET PENSJON (AFP) FOR STATLIG OG KOMM...,Arbeidsforhold
4,BARNEBIDRAG,Andre økonomiske opplysninger


In [650]:
for index, row in df_current_purpose.iterrows():
    delete_url = f"{codelist_url}/category/{row['index']}"
    requests.delete(delete_url)

### Populate categories

In [651]:
xls = pd.ExcelFile('../data/testdata.xlsx')
sheet=xls.sheet_names[5]
df_categories = pd.read_excel(xls, sheet)
df_categories = df_categories[['Code','Description']]
df_categories.columns = ['code','description']
df_categories['code'] = df_formål['code'].apply(lambda x: x.upper())
df_categories['list']='category'
df_categories = df_categories.append({'code': 'NOT AVAILABLE', 'description': 'Not Available -Verdi mangler', 'list': 'category'}, ignore_index=True)
df_categories.head()

,code,description,list
0,ALDERSPENSJON,Motorvognopplysninger,category
1,ARBEIDSAVKLARINGSPENGER (AAP),Opplysninger om reisevei,category
2,AVTALEFESTET PENSJON (AFP) FOR PRIVAT SEKTOR,Adresse og kontaktopplysninger,category
3,AVTALEFESTET PENSJON (AFP) FOR STATLIG OG KOMM...,Arbeidsforhold,category
4,BARNEBIDRAG,Andre økonomiske opplysninger,category


In [652]:
data = df_categories.to_json(orient='records')
payload =json.dumps(json.loads(data))
requests.post(codelist_url, headers=headers, data=payload)

<Response [201]>

## Provenance

### Delete existing

In [823]:
df_current_provenance = df_codelist[['index','PROVENANCE']].dropna()
df_current_provenance.head()

,index,PROVENANCE


### Repopulate

In [824]:
provenance = """{"provenance": {
       "UTLENDINGSDIREKTORATET": "Utlendingsdirektoratet",
       "ARBEIDSGIVER": "Arbeidsgiver",
       "FOLKEREGISTERET": "Folkeregisteret",
       "SKATTEETATEN": "Skatteetaten",
       "BRUKER": "Informasjon oppgitt av bruker",
       "UTENLANDS_TRYGDEMYNDIGHET": "Utenlands trygdemyndighet",
       "REVISOR": "Revisor"
   }}"""

In [825]:
df_provenance = pd.read_json(json.dumps(json.loads(provenance))).reset_index()
df_provenance.columns = ['code','description']
df_provenance['code'] = df_provenance['code'].apply(lambda x: x.upper())
df_provenance['list']='provenance'

In [826]:
df_provenance.head()

,code,description,list
0,ARBEIDSGIVER,Arbeidsgiver,provenance
1,BRUKER,Informasjon oppgitt av bruker,provenance
2,FOLKEREGISTERET,Folkeregisteret,provenance
3,REVISOR,Revisor,provenance
4,SKATTEETATEN,Skatteetaten,provenance


In [827]:
data = df_provenance.to_json(orient='records')
data

'[{"code":"ARBEIDSGIVER","description":"Arbeidsgiver","list":"provenance"},{"code":"BRUKER","description":"Informasjon oppgitt av bruker","list":"provenance"},{"code":"FOLKEREGISTERET","description":"Folkeregisteret","list":"provenance"},{"code":"REVISOR","description":"Revisor","list":"provenance"},{"code":"SKATTEETATEN","description":"Skatteetaten","list":"provenance"},{"code":"UTENLANDS_TRYGDEMYNDIGHET","description":"Utenlands trygdemyndighet","list":"provenance"},{"code":"UTLENDINGSDIREKTORATET","description":"Utlendingsdirektoratet","list":"provenance"}]'

In [830]:
payload =json.dumps(json.loads(data))
requests.post(codelist_url, headers={'Content-Type': 'application/json'}, data=payload)

<Response [201]>

# Datasett

In [ ]:
datasett_url = policy_url = 'https://datakatalog.nais.oera-q.local/policies/datasett'

In [499]:
sheet=xls.sheet_names[3]
df_datasett = pd.read_excel(xls, sheet)
df_datasett.head(1)

,"""name"": ""","""description"": ""","""categoryCode"": ""","""producerCode"": [""","""systemCode"": ""","""personalData"": """,Kommentar,JSON
0,Sivilstand,En overordnet kategori som beskriver en person...,PERSONALIA,FOLKEREGISTERET,TPS,True,NaN,"{ \n""name"": ""Sivilstand"",\n""description"": ""En ..."


# Policies

### Delete policies

In [840]:
policy_url = 'https://datakatalog.nais.oera-q.local/policies/policy'

In [841]:
url = 'https://datakatalog.nais.oera-q.local/policies/policy?pageNumber=0&pageSize=1000'
print(url)
df_policies_current = pd.read_json(url)

https://datakatalog.nais.oera-q.local/policies/policy?pageNumber=0&pageSize=1000


In [842]:
df_policies_current

,content,currentPage,pageSize,totalElements
0,"{'policyId': 236, 'dataset': {'id': 'c30ac412-...",0,1000,4
1,"{'policyId': 237, 'dataset': {'id': 'c30ac412-...",0,1000,4
2,"{'policyId': 238, 'dataset': {'id': 'c30ac412-...",0,1000,4
3,"{'policyId': 239, 'dataset': {'id': 'c30ac412-...",0,1000,4


In [843]:
print(len(df_policies_current))

for index, row in df_policies_current.iterrows():
    policy_id = row['content']['policyId']
    url = f"{policy_url}/{policy_id}"
    print(url)
    requests.delete(url, headers = {'accept': '*/*'})

4
https://datakatalog.nais.oera-q.local/policies/policy/236
https://datakatalog.nais.oera-q.local/policies/policy/237
https://datakatalog.nais.oera-q.local/policies/policy/238
https://datakatalog.nais.oera-q.local/policies/policy/239


In [844]:
df_policies_content = df_policies_current['content']

In [845]:
xls = pd.ExcelFile('../data/testdata.xlsx')
sheet=xls.sheet_names[4]
df_policies = pd.read_excel(xls, sheet)
df_policies.head(1)

,"""informationTypeName"": ""","""legalBasisDescription"": ""","""purposeCode"": """,Detaljering av purpose (mulighet til å legge inn tekstlig beskrivelse av formålet utover selve purposeCode. Må ligge i tabellen POLICY. F.eks. POLICY.DESCRIPTION ),Kommentar,Generer JSON,JSON
0,Sivilstand,Ftrl. § 11-20,Arbeidsavklaringspenger (AAP),Ved nytt krav og nytt barn,NaN,Ja,"{ ""informationTypeName"": ""Sivilstand"", \n""purp..."


In [846]:
df_policies = df_policies[['"informationTypeName": "','"purposeCode": "','Detaljering av purpose (mulighet til å legge inn tekstlig beskrivelse av formålet utover selve purposeCode. Må ligge i tabellen POLICY. F.eks. POLICY.DESCRIPTION )' ]]

In [847]:
def getID(row):
    return str(uuid.uuid4())

df_policies.columns=['datasetTitle','purposeCode','legalBasisDescription']
df_policies['id'] =  range(len(df_policies)) #df_policies.apply(getID, axis=1)

In [848]:
df_policies.head()

,datasetTitle,purposeCode,legalBasisDescription,id
0,Sivilstand,Arbeidsavklaringspenger (AAP),Ved nytt krav og nytt barn,0
1,Sivilstand,Alderspensjon,Har betyding for pensjonsberegningen,1
2,Sivilstand,Krigspensjon,Har betyding for pensjonsberegningen,2
3,Sivilstand,Avtalefestet pensjon (AFP) for statlig og komm...,Vurdering av sivilstand,3
4,Sivilstand,Barnetrygd,knyttet til vilkåret for barnetrygd,4


In [849]:
df_policies.fillna('NOT AVAILABLE', inplace=True)

In [850]:
df_policies[df_policies['datasetTitle'] =='Sivilstand']

,datasetTitle,purposeCode,legalBasisDescription,id
0,Sivilstand,Arbeidsavklaringspenger (AAP),Ved nytt krav og nytt barn,0
1,Sivilstand,Alderspensjon,Har betyding for pensjonsberegningen,1
2,Sivilstand,Krigspensjon,Har betyding for pensjonsberegningen,2
3,Sivilstand,Avtalefestet pensjon (AFP) for statlig og komm...,Vurdering av sivilstand,3
4,Sivilstand,Barnetrygd,knyttet til vilkåret for barnetrygd,4
5,Sivilstand,Kontantstøtte,knyttet til vilkåret for Kontantstøtte,5
6,Sivilstand,Barnebidrag,Behovsprøving av forskudd. førstegangsfastsett...,6
7,Sivilstand,Dagpenger,Vurdering å kunne ansees som reell arbeidssøker,7
8,Sivilstand,Ektefellebidrag,Vuderinga av søknad/ endring utmåling og opph...,8
9,Sivilstand,Stønad til enslig mor eller far,"Vurdere vilkår om sivilstatus (ugift, separert...",9


In [856]:
data = df_policies[0:20].to_json(orient='records')
payload =json.dumps(json.loads(data))
print(policy_url)
print(payload)
requests.post(policy_url, headers={'Content-Type': 'application/json'}, data=payload)

https://datakatalog.nais.oera-q.local/policies/policy
[{"datasetTitle": "Sivilstand", "purposeCode": "Arbeidsavklaringspenger (AAP)", "legalBasisDescription": "Ved nytt krav og nytt barn", "id": 0}, {"datasetTitle": "Sivilstand", "purposeCode": "Alderspensjon", "legalBasisDescription": "Har betyding for pensjonsberegningen", "id": 1}, {"datasetTitle": "Sivilstand", "purposeCode": "Krigspensjon", "legalBasisDescription": "Har betyding for pensjonsberegningen", "id": 2}, {"datasetTitle": "Sivilstand", "purposeCode": "Avtalefestet pensjon (AFP) for statlig og kommunal sektor", "legalBasisDescription": "Vurdering av sivilstand", "id": 3}, {"datasetTitle": "Sivilstand", "purposeCode": "Barnetrygd", "legalBasisDescription": "knyttet til vilk\u00e5ret for barnetrygd", "id": 4}, {"datasetTitle": "Sivilstand", "purposeCode": "Kontantst\u00f8tte", "legalBasisDescription": "knyttet til vilk\u00e5ret for Kontantst\u00f8tte", "id": 5}, {"datasetTitle": "Sivilstand", "purposeCode": "Barnebidrag", "l

<Response [400]>

In [695]:
data = df_policies.head(1).to_json(orient='records')
payload =json.dumps(json.loads(data))
try:
    res = requests.post(policy_url, headers=headers, data=payload)
except:
    res = payload
    
print(res)

<Response [201]>


In [852]:
data = df_policies.to_json(orient='records')
payload =json.dumps(json.loads(data))

In [853]:
requests.post(url, headers={'Content-Type': 'application/json'}, data=payload)

<Response [405]>